Machine Learning for Churn Classification

In [687]:
import pandas as pd
import numpy as np

DATA PREPARATION

In [688]:
# to get access to the csv file
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [689]:
# to get the data
!wget $data

--2025-10-21 23:19:30--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.22’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.002s  

2025-10-21 23:19:30 (43.0 MB/s) - ‘course_lead_scoring.csv.22’ saved [80876/80876]



In [690]:
# to put the data on dataframe
df = pd.read_csv('course_lead_scoring.csv').convert_dtypes()

In [691]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,<NA>,1,79450,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992,employed,south_america,1,0.8,0
2,events,healthcare,5,78796,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843,<NA>,australia,1,0.87,0
4,referral,education,3,85012,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,<NA>,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688,student,north_america,3,0.02,1
1460,referral,<NA>,5,71016,self_employed,north_america,0,0.25,1


In [692]:
# to have a look at the first five row
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,<NA>,1,79450,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992,employed,south_america,1,0.8,0
2,events,healthcare,5,78796,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843,<NA>,australia,1,0.87,0
4,referral,education,3,85012,self_employed,europe,3,0.62,1


In [693]:
# to transpose the table so that the column will be well position for clarity
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,<NA>,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450,46992,78796,83843,85012
employment_status,unemployed,employed,unemployed,<NA>,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [694]:
# to check for the last row in the table
df.tail()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1457,referral,manufacturing,1,<NA>,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688,student,north_america,3,0.02,1
1460,referral,<NA>,5,71016,self_employed,north_america,0,0.25,1
1461,organic_search,finance,3,92855,student,north_america,3,0.41,1


In [695]:
# to see the cloumns in the dataframe
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [696]:
# to see the data types of the columns
df.dtypes

lead_source                 string[python]
industry                    string[python]
number_of_courses_viewed             Int64
annual_income                        Int64
employment_status           string[python]
location                    string[python]
interaction_count                    Int64
lead_score                         Float64
converted                            Int64
dtype: object

In [697]:
# to see those that were object explicit;y
df.dtypes == "string"

lead_source                  True
industry                     True
number_of_courses_viewed    False
annual_income               False
employment_status            True
location                     True
interaction_count           False
lead_score                  False
converted                   False
dtype: bool

In [698]:
categorical_columns = df.dtypes[df.dtypes == "string"].index

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(" ", "_")



In [699]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   string 
 1   industry                  1328 non-null   string 
 2   number_of_courses_viewed  1462 non-null   Int64  
 3   annual_income             1281 non-null   Int64  
 4   employment_status         1362 non-null   string 
 5   location                  1399 non-null   string 
 6   interaction_count         1462 non-null   Int64  
 7   lead_score                1462 non-null   Float64
 8   converted                 1462 non-null   Int64  
dtypes: Float64(1), Int64(4), string(4)
memory usage: 110.1 KB


In [700]:
df.count()

lead_source                 1334
industry                    1328
number_of_courses_viewed    1462
annual_income               1281
employment_status           1362
location                    1399
interaction_count           1462
lead_score                  1462
converted                   1462
dtype: int64

In [701]:
df.loc[df.isna().any(axis = "columns")]

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,<NA>,1,79450,unemployed,south_america,4,0.94,1
3,paid_ads,retail,2,83843,<NA>,australia,1,0.87,0
5,events,manufacturing,1,59904,<NA>,africa,6,0.83,1
6,social_media,technology,0,51283,<NA>,middle_east,2,0.57,0
7,social_media,<NA>,5,62975,student,europe,4,0.62,1
...,...,...,...,...,...,...,...,...,...
1454,referral,finance,0,75546,<NA>,asia,0,0.1,0
1455,referral,finance,2,<NA>,<NA>,south_america,3,0.47,1
1456,social_media,healthcare,1,45253,<NA>,australia,6,0.24,1
1457,referral,manufacturing,1,<NA>,self_employed,north_america,4,0.53,1


In [702]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [703]:
df = df.bfill()

In [704]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,retail,1,79450,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992,employed,south_america,1,0.8,0
2,events,healthcare,5,78796,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843,self_employed,australia,1,0.87,0
4,referral,education,3,85012,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,65259,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688,student,north_america,3,0.02,1
1460,referral,finance,5,71016,self_employed,north_america,0,0.25,1


In [705]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [706]:
df.isnull().sum().sum()

np.int64(0)

In [709]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,retail,1,79450,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992,employed,south_america,1,0.8,0
2,events,healthcare,5,78796,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843,self_employed,australia,1,0.87,0
4,referral,education,3,85012,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,65259,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688,student,north_america,3,0.02,1
1460,referral,finance,5,71016,self_employed,north_america,0,0.25,1


In [710]:
df.industry.shape

(1462,)

In [711]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,retail,1,79450,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992,employed,south_america,1,0.8,0
2,events,healthcare,5,78796,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843,self_employed,australia,1,0.87,0
4,referral,education,3,85012,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,65259,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688,student,north_america,3,0.02,1
1460,referral,finance,5,71016,self_employed,north_america,0,0.25,1


In [712]:
df = df.to_csv('course_lead_scoring_cleansed.csv', index = False)

In [713]:
df = pd.read_csv('course_lead_scoring_cleansed.csv').convert_dtypes()

In [714]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   string 
 1   industry                  1462 non-null   string 
 2   number_of_courses_viewed  1462 non-null   Int64  
 3   annual_income             1462 non-null   Int64  
 4   employment_status         1462 non-null   string 
 5   location                  1462 non-null   string 
 6   interaction_count         1462 non-null   Int64  
 7   lead_score                1462 non-null   Float64
 8   converted                 1462 non-null   Int64  
dtypes: Float64(1), Int64(4), string(4)
memory usage: 110.1 KB


In [715]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,retail,1,79450,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992,employed,south_america,1,0.8,0
2,events,healthcare,5,78796,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843,self_employed,australia,1,0.87,0
4,referral,education,3,85012,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,65259,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688,student,north_america,3,0.02,1
1460,referral,finance,5,71016,self_employed,north_america,0,0.25,1


In [716]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [718]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

dv = DictVectorizer(sparse=False)

y_full_train = df_full_train.converted.values
y_test = df_test.converted.values
y_train = df_train.converted.values
y_val = df_val.converted.values

dicts_train = df_train.drop(columns="converted").to_dict(orient="records")
dicts_val = df_val.drop(columns="converted").to_dict(orient="records")

dv.fit(dicts_train)
X_train = dv.transform(dicts_train)
X_val = dv.transform(dicts_val)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)

(y_val==converted_decision).mean()

accuracy_score(y_val, converted_decision)

0.7440273037542662

In [722]:
#use model to predict probability that given y value is 1
y_pred_proba = LogisticRegression.predict_proba(X_test)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)

0.5602104030579559

TypeError: LogisticRegression.predict_proba() missing 1 required positional argument: 'X'